<a href="https://colab.research.google.com/github/dapsavoie/tz_water_well_prediction/blob/master/37_Submitted_Score_Class_Imbalance_after_Dummies.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd 
import numpy as np

In [246]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
#Train dummies 
train_dummies = pd.read_csv(r'/content/drive/My Drive/tz_water_well_data/train_dummmies.csv')

#Test export to csv 
test_dummies = pd.read_csv(r'/content/drive/My Drive/tz_water_well_data/test_dummies.csv')

#df_labels
df_labels = pd.read_csv('/content/drive/My Drive/tz_water_well_data/training_set_values.csv')

In [0]:
y = df_labels['status_group']

X = train_dummies

In [249]:
train_dummies.shape

(59400, 316)

# Implementing oversampling for handling imbalanced data

In [0]:
from imblearn.over_sampling import SMOTE, ADASYN

In [251]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=123)

X_train_resampled, y_train_resampled = SMOTE().fit_sample(X_train, y_train) 

# Preview synthetic sample class distribution
print('\n')
print(pd.Series(y_train_resampled).value_counts()) 

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)




functional needs repair    24296
functional                 24296
non functional             24296
dtype: int64


In [252]:
X_train_resampled.shape, y_train_resampled.shape

((72888, 316), (72888,))

# Fit the Model on SMOTE Data

In [0]:
import xgboost as xgb
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

In [0]:
scaler = StandardScaler()

In [0]:
scaled_df = scaler.fit_transform(X_train_resampled)

In [0]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_train_resampled, y_train_resampled)

In [257]:
#Train

clf = xgb.XGBClassifier()
clf.fit(X_train, y_train)
training_preds = clf.predict(X_train)
val_preds = clf.predict(X_test)
training_accuracy = accuracy_score(y_train, training_preds)
val_accuracy = accuracy_score(y_test, val_preds)

print("Training Accuracy: {:.4}%".format(training_accuracy * 100))
print("Validation accuracy: {:.4}%".format(val_accuracy * 100))

Training Accuracy: 80.53%
Validation accuracy: 80.04%


In [258]:
from sklearn.metrics import recall_score

recall_score(y_test, val_preds, average='micro')*100

80.04061025134452

In [259]:
test_preds = clf.predict(test_dummies, validate_features=False)
test_preds

array(['functional', 'functional', 'non functional', ..., 'functional',
       'functional', 'non functional'], dtype=object)

In [260]:
test_preds.shape, val_preds.shape, test_dummies.shape, y_test.shape,training_preds.shape

((14850,), (18222,), (14850, 316), (18222,), (54666,))

In [0]:
scaled_df = scaler.fit_transform(test_dummies)

In [0]:
test_preds = clf.predict(scaled_df, validate_features=False)

In [0]:
test_preds = pd.DataFrame(test_preds)

In [0]:
predictions = test_preds
y_test = pd.DataFrame(test_dummies['id'])
submission = pd.DataFrame(y_test.join(test_preds))

In [0]:
submission.rename(columns={0: 'status_group'}, inplace=True)

In [268]:
submission.columns

Index(['id', 'status_group'], dtype='object')

In [0]:
submission.to_csv('/content/drive/My Drive/tz_water_well_data/jesu.csv' ,index=0)